In [ ]:
#most codes were either modified codes made by Yang Li, or modified cantera codes with the help of Yang Li

In [ ]:
#The purpose of these codes is to be able to simulate the outcomes of the steam methane reforming for a variety of conditions

In [ ]:
#installing the cantera package needed for chemical kinetic calculations
 !pip install cantera

#additional package needed
import cantera as ct
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 12.3 MB/s eta 0:00:00


In [ ]:
# import drive for data files
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
#compute the density of the gas for a single, constant temperature
def compute_density(amount, temperature):
    """This functions returns the gas density in kg/m^3 for a gas with methane-to-water mixing ratio of 1:x with x being set by the argument "amount", and the gas temperature has been defined by the argument "temperature".
    Temperature must be a single value."""

    #temperature
    T_0_1 = temperature  # inlet temperature [K]

    #pressure
    pressure = ct.one_atm  # constant pressure [Pa]

    #initial gas composition
    composition_0 = f'CH4:1, H2O:{amount}'

    # input file containing the reaction mechanism
    reaction_mechanism = 'gri30.yaml'

    # import the gas model and set the initial conditions
    gas1 = ct.Solution(reaction_mechanism)
    gas1.TPX = T_0_1, pressure, composition_0

    #safe gas density
    density = gas1.density

    return density #kg/m^3


#define the intial density of the gas.
def compute_unevenTemp_density(amount, temperature):
    """This functions returns the gas density in kg/m^3 for a gas with methane-to-water mixing ratio of 1:x with x being set by the argument "amount", and the gas temperature has been defined by the argument "temperature".
    Temperature must be a list."""

    #retrieve initial temperature
    T_0_1 = temperature[0]  # inlet temperature [K]

    #set pressure
    pressure = ct.one_atm  # constant pressure [Pa]

    #set initial gas composition
    composition_0 = f'CH4:1, H2O:{amount}'

    # input file containing the reaction mechanism
    reaction_mechanism = 'gri30.yaml'

    # import the gas model and set the initial conditions
    gas1 = ct.Solution(reaction_mechanism)
    gas1.TPX = T_0_1, pressure, composition_0

    #safe gas density
    density = gas1.density

    return density #kg/m^3

def flow_initial_velocity_calculator(molar_flow, composition, density, radius):
    """The function returns the intial flow velocity of the gas when the molar flow is given in kmol/h, the initial methane-to-water mixing ratio 1:x with x being set by the argument "composition", the gas density in kg/m^3, and
    the inner radius of the tube in meters have been supplied as arguments"""

    #calculate the crossectional area of the tube in
    area= np.pi*radius**2 #m^2

    #Molar flow calculations
    total_molar_flow = molar_flow #kmol/h
    CH4_molar_flow = total_molar_flow/(composition+1) #kmol/h
    H2O_molar_flow = composition*CH4_molar_flow #kmol/h

    #calculate mass flows
    CH4_mass_flow = (12.01+1.008*4)*CH4_molar_flow/3600 #kg/s
    H2O_mass_flow = (1.008*2+16)*H2O_molar_flow/3600 #kg/s
    total_mass_flow = H2O_mass_flow+CH4_mass_flow

    #calculate flow velocity
    U = total_mass_flow/(density*area) #m/s
    return U

In [ ]:
#approach used
#these codes are based on the chain of reactor model written given by cantera and were modified with the help of Yang Li
#(D. G. Goodwin, H. K. Moffat, I. Schoegl, R. L. Speth, and B. W. Weber, Cantera: An object-oriented software toolkit for chemical kinetics, thermodynamics, and transport processes, https://www.cantera.org, Version 3.0.0, 2023. DOI: 10.5281/zenodo.8137090.)

def compute_lengthbased_array_wallbasedtemperature(depth, amount, initial_gas_temperature, wall_temperature, flow_velocity, radius):
    """Computes the gas states of a gas within a plug flow reactor connected to a heat reservoir. When the axial position (depth) in the form of an array, the amount of water in the methane-to-water mixing ratio of 1:{amount}
    initial gas temperature, temperature profile of the wall in the form of an array, initial flow velocity of the gas
    and the inner tube radius are supplied as arguments"""

    #rename values
    length= depth
    u_0=flow_velocity

    #calculate cross sectional area of the tube
    cross_section = np.pi*radius**2

    #calculate the stepsize between two axial positions
    dz = length[-1] / len(length)

    #calculate the unit volume for each individual step
    volume = dz*cross_section
    #calculate the area of the wall for each individual step
    area = 2*np.pi*radius*dz

    #create empty arras to store values
    #total time
    t = np.zeros(len(length))
    #flow velocity
    u1=np.zeros(len(length))
    #residence time per unit volume
    t_r2 = np.zeros(len(length))
    #Reynolds number
    Reynolds = np.zeros(len(length))

    #change name temperature
    T_0_1 = initial_gas_temperature  # inlet temperature [K]

    #set pressure
    pressure = ct.one_atm  # constant pressure [Pa]

    #define initial gas mixture
    composition_0 = f'CH4:1, H2O:{amount}'

    # input file containing the reaction mechanism
    reaction_mechanism = 'gri30.yaml'

    # import the gas model and set the initial conditions
    gas1 = ct.Solution(reaction_mechanism)
    gas1.TPX = T_0_1, pressure, composition_0
    mass_flow_rate1 = u_0 * gas1.density * cross_section

    # create a new reactor
    r1 = ct.IdealGasReactor(gas1, energy='on')
    r1.volume = volume

    #create upstream and downstream reactors
    upstream = ct.Reservoir(gas1, name='upstream')
    downstream = ct.Reservoir(gas1, name='downstream')

    #mass controller
    m = ct.MassFlowController(upstream, r1, mdot=mass_flow_rate1)

    #pressure controller
    v = ct.PressureController(r1, downstream, primary=m, K=1e-5)

    # create a reactor network for performing time integration
    sim1 = ct.ReactorNet([r1])

    #tolerance values
    sim1.rtol = 1e-8
    sim1.atol = 1e-15

    # define time, space, and other information vectors
    states1 = ct.SolutionArray(r1.thermo, extra=['U', 'Re', 'tres'])

    Temp = initial_gas_temperature

    #heat coefficient
    lamda = gas1.thermal_conductivity
    heat_transfer_coefficient = lamda/radius

    #wall and reservoir
    air = ct.Solution('air.yaml')
    air.TP = wall_temperature[0], None
    wall_reservoir = ct.Reservoir(air)
    wall = ct.Wall(r1, wall_reservoir, A=area, U=heat_transfer_coefficient)

    for n in range(len(length)):
        # Set the state of the reservoir to match that of the previous reactor
        gas1.TDY = r1.thermo.TDY
        upstream.syncState()
        r1.syncState()

        #recalculate thermal conductivity
        lamda = gas1.thermal_conductivity

        #recalculate heat_transfer_coefficient
        if length[n] == 0:
            heat_transfer_coefficient = lamda/radius
        else:
            heat_transfer_coefficient = Nusselt(Graetz(length[n], gas1.density, mass_flow_rate1 / cross_section / r1.thermo.density, gas1.cp_mass, radius, lamda))*lamda / (2*radius)
        wall.heat_transfer_coeff = heat_transfer_coefficient

        #integrate to steady state
        sim1.reinitialize()
        sim1.set_initial_time(0)
        sim1.advance_to_steady_state()

        # compute velocity and transform into time
        u1[n] = mass_flow_rate1 / cross_section / r1.thermo.density
        t_r2[n] = r1.mass / mass_flow_rate1  # residence time in this reactor #using this no reaction obtained
        t[n] = np.sum(t_r2) #reaction obtained using this, does this make sense?

        #Calculate reynolds number for verification if laminar flow is indeed appropriate
        Reynolds[n] = Reynold(gas1.density, gas1.viscosity, radius, u1[n])

        # write output data
        states1.append(r1.thermo.state, U=u1[n], Re=Reynolds[n], tres=t_r2[n])

        #redefine wall temperature
        air.TP = wall_temperature[n], None
        wall_reservoir.syncState()

    return states1

In [ ]:
def simulated_temperature_profile(x, a, b, A, B, E):
    """ function that can be used as temperature profile for axial position x defined by an array, and parameters a,b,A,B,E"""

    #create empty list to store values
    function = []

    #temperature function has the form of an error function
    for i in x:
        if i <= (a+b)/2:
            function.append(math.erf(E*(i-a))*A/2+B+A/2)
        if i>= (a+b)/2:
             function.append(-math.erf(E*(i-b))*A/2+B+A/2)

    #change the list into an array
    function = np.array(function)

    return function

def simulated_temperature(x, a, b, A, B, E):
    """function that can be used to return a single temperature for a single axial position x and parameters a,b,A,B,E"""

    #Temperature calculation
    if x <= (a+b)/2:
        T=math.erf(E*(x-a))*A/2+B+A/2
    if x >= (a+b)/2:
        T=-math.erf(E*(x-b))*A/2+B+A/2

    return T

In [ ]:
#core code supplied by Yang Li and then modified
def compute_timebased_array(time, element, amount, temperature):
    """This function returns the molefraction of a substance present in a gas within a zero dimensional reactor when the time is given as an array, the element is the chemical notation of the substance of interest,
    amount specifies the amount of water molecules initially present in the gas, and a single constant temperature"""

    #Temperature
    T_0_1 = temperature  # inlet temperature [K]

    pressure = ct.one_atm  # constant pressure [Pa]

    #initial gas composition
    composition_0 = f'CH4:1, H2O:{amount}'

    # input file containing the reaction mechanism
    reaction_mechanism = 'gri30.yaml'

    # import the gas model and set the initial conditions
    gas1 = ct.Solution(reaction_mechanism)
    gas1.TPX = T_0_1, pressure, composition_0

    # create a new reactor
    r1 = ct.IdealGasConstPressureReactor(gas1, energy='off')

    # create a reactor network for performing time integration
    sim1 = ct.ReactorNet([r1])

    states1 = ct.SolutionArray(r1.thermo)

    for n1, t_i in enumerate(time):
        # perform time integration
        sim1.advance(t_i)
        states1.append(r1.thermo.state)

    #return gas mole fractions
    timed_concentration = states1(element).X

    return timed_concentration

In [ ]:
# example code zero dimensional reactor

#define time array
t= np.linspace(0, 1000, 10000)

#Hydrogen mole fraction plotted against time for a set of temperatures
plt.plot(t, compute_timebased_array(t, 'H2', 2, 800), label='T=800 K')
plt.plot(t, compute_timebased_array(t, 'H2', 2, 1000), label='T=1000 K')
plt.plot(t, compute_timebased_array(t, 'H2', 2, 1200), label='T=1200 K')
plt.plot(t, compute_timebased_array(t, 'H2', 2, 1400), label='T=1400 K')
plt.plot(t, compute_timebased_array(t, 'H2', 2, 1600), label='T=1600 K')

plt.legend(loc='best')
plt.title("H2 mole fractions, Ch4+2H2O, Chemical Kinetics")
plt.ylabel('mole fraction')
plt.xlabel('time (s)')

In [ ]:
#example code plug flow reactor

#temperature furnace
data = np.genfromtxt('path_to_data_set', delimiter=';', skip_header=1)

# Extract data from each column into separate arrays
elevation = data[:, 0]  # First column
Temp = data[:, 1]  # Second column

#initial guess for better accuracy
initial_guess = [0.2, 1.2, 700, 500, 100]

#retrieve parameters for a temperature fit
params, covariance = scipy.optimize.curve_fit(simulated_temperature_profile, elevation, Temp, p0=initial_guess)
a_fit, b_fit, A_fit, B_fit, E_fit = params

#define the same axial positions but with more inbetween steps
z = np.linspace(0, elevation[-1], 10000)

#offset the fit for a higher temperature profile
new_Temp = simulated_temperature_profile(z, a_fit, b_fit, A_fit+300, B_fit, E_fit)

#states01 = compute_lengthbased_array_wallbasedtemperature(z, 2, new_Temp[0], new_Temp, flow_initial_velocity_calculator(0.1*0.14342719311951801*10**(-3), 2, compute_density(2, new_Temp[0]), 0.006), 0.006)
states05 = compute_lengthbased_array_wallbasedtemperature(z, 2, new_Temp[0], new_Temp, flow_initial_velocity_calculator(0.5*0.14342719311951801*10**(-3), 2, compute_density(2, new_Temp[0]), 0.006), 0.006)
states1 = compute_lengthbased_array_wallbasedtemperature(z, 2, new_Temp[0], new_Temp, flow_initial_velocity_calculator(1*0.14342719311951801*10**(-3), 2, compute_density(2, new_Temp[0]), 0.006), 0.006)
states2 = compute_lengthbased_array_wallbasedtemperature(z, 2, new_Temp[0], new_Temp, flow_initial_velocity_calculator(2*0.14342719311951801*10**(-3), 2, compute_density(2, new_Temp[0]), 0.006), 0.006)
states10 = compute_lengthbased_array_wallbasedtemperature(z, 2, new_Temp[0], new_Temp, flow_initial_velocity_calculator(10*0.14342719311951801*10**(-3), 2, compute_density(2, new_Temp[0]), 0.006), 0.006)
states100 = compute_lengthbased_array_wallbasedtemperature(z, 2, new_Temp[0], new_Temp, flow_initial_velocity_calculator(100*0.14342719311951801*10**(-3), 2, compute_density(2, new_Temp[0]), 0.006), 0.006)


#plot hydrogen mole fraction against lenght
plt.plot(z, states05('H2').X, label=r'$0.5 \dot{m}$')
plt.plot(z, states1('H2').X, label=r'$ \dot{m}$')
plt.plot(z, states2('H2').X, label=r'$2 \dot{m}$')
plt.plot(z, states10('H2').X, label=r'$10 \dot{m}$')
plt.plot(z, states100('H2').X, label=r'$100 \dot{m}$')

plt.title("H2, furnace offsetted")
plt.legend(loc='best')
plt.ylabel('mole fraction')
plt.xlabel('length (m)')
plt.xlim(0, 1.4)
plt.show()